In [1]:
import cdsapi
import requests
import os

In [ ]:
#What a permaculture project need to know when they start

In [34]:
import time
from geopy.geocoders import Nominatim

# select the years you want to download:
start_year = 2004
end_year = 2006
year_range = [i for i in range(start_year, end_year + 1)]
print(year_range)

# Use geopy to get the latitude and longitude of the city
geolocator = Nominatim(user_agent="permaculture-climate")
location = geolocator.geocode("Puebla de don Fadrique, Spain")
# Add a delay between requests
time.sleep(1)

c = cdsapi.Client()
try:
    data = c.retrieve("reanalysis-era5-single-levels-monthly-means",
    {"format": "grib",
     "product_type": "monthly_averaged_reanalysis_by_hour_of_day",
     "variable": ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                '2m_temperature',
                #'snowfall', 
                'soil_temperature_level_1', 
                'soil_type',
                'total_cloud_cover', 
                'total_precipitation',
                ],
    "area": [location.latitude + 1, 
             location.longitude - 1, 
             location.latitude - 1, 
             location.longitude + 1],  # North, West, South, East. 
    "year": year_range,
    "month": ['01', '02', '03',
           '04', '05', '06',
           '07', '08', '09',
           '10', '11', '12'],
    "time": ["00:00","01:00","02:00","03:00","04:00","05:00",
             "06:00","07:00","08:00","09:00","10:00","11:00",
             "12:00", "13:00","14:00","15:00","16:00","17:00",
             "18:00","19:00","20:00","21:00","22:00","23:00"]
    })

    # Get the location of the file to download
    url = data.location

    # Download the file
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

except requests.exceptions.HTTPError as errh:
    print ("HTTP Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("Something went wrong with the request:",err)

else:
    # If the request was successful, write the file
    filename = 'download.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

    # Print the location where the file is saved
    print(f"File saved at: {os.path.abspath(filename)}")

[2004, 2005, 2006]


2023-11-22 12:49:40,685 INFO Welcome to the CDS
2023-11-22 12:49:40,686 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-11-22 12:49:40,784 INFO Request is queued
2023-11-22 12:51:34,732 INFO Request is running
2023-11-22 12:57:59,501 INFO Request is completed


File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download.grib


In [ ]:
#temperatura durante l'anno 
#precipitaciones durante l'anno
#vento durante l'anno (rosa dei venti)

In [3]:
#use pygrib to read the variables names
import pygrib

# Open the GRIB file
grbs = pygrib.open('download.grib')

# Read the names of the variables
variable_names = [grb.name for grb in grbs]

# Convert the set to a list
unique_variable_names = list(set(variable_names))

# Print the variable names
print(unique_variable_names)


['Soil type', 'Soil temperature level 1', '2 metre temperature', 'Total precipitation', '10 metre U wind component', 'Total cloud cover', '10 metre V wind component']


In [27]:
import xarray as xr

# List of variables to load
variables = ['Total cloud cover', 'Soil temperature level 1', '10 metre V wind component', 
             '2 metre temperature', 'Soil type', 'Total precipitation', '10 metre U wind component']

# Dictionary to hold the datasets
datasets = {}

# Open the GRIB file for each variable
for var in variables:
    ds = xr.open_dataset('download.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'parameterName': var}})
    datasets[var] = ds

# Print the datasets
for var, ds in datasets.items():
    print(f"{var}:")
    print(ds)
    print("\n")


Ignoring index file 'download.grib.dc8f2.idx' older than GRIB file
Ignoring index file 'download.grib.dc8f2.idx' older than GRIB file
Ignoring index file 'download.grib.dc8f2.idx' older than GRIB file
Ignoring index file 'download.grib.dc8f2.idx' older than GRIB file
Ignoring index file 'download.grib.dc8f2.idx' older than GRIB file
Ignoring index file 'download.grib.dc8f2.idx' older than GRIB file
Ignoring index file 'download.grib.dc8f2.idx' older than GRIB file


Total cloud cover:
<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    Conventions:  CF-1.7
    history:      2023-11-22T19:05 GRIB to CDM+CF via cfgrib-0.9.10.4/ecCodes...


Soil temperature level 1:
<xarray.Dataset>
Dimensions:              (time: 864, latitude: 9, longitude: 9)
Coordinates:
    number               int64 ...
  * time                 (time) datetime64[ns] 2004-01-01 ... 2006-12-01T23:0...
    step                 timedelta64[ns] ...
    depthBelowLandLayer  float64 ...
  * latitude             (latitude) float64 38.96 38.71 38.46 ... 37.21 36.96
  * longitude            (longitude) float64 -3.435 -3.185 ... -1.684 358.6
    valid_time           (time) datetime64[ns] ...
Data variables:
    stl1                 (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conv

In [ ]:
#check the diemsnsions of the data
print(temp_climatology.dims)
print(precip_climatology.dims)

In [36]:
import plotly.graph_objects as go

# Calculate the climatology and average over latitude and longitude
temp_climatology = datasets['2 metre temperature']['t2m'].groupby('time.month').mean(['time', 'latitude', 'longitude'])-273.15
precip_climatology = datasets['Total precipitation']['tp'].groupby('time.month').mean(['time', 'latitude', 'longitude', 'step'])*1000

#check the diemsnsions of the data
print(temp_climatology.dims)
print(precip_climatology.dims)


('month',)
('month',)


In [38]:
# Create a Plotly figure
fig = go.Figure()

# Add a bar chart for precipitation
fig.add_trace(go.Bar(
    x=precip_climatology.month,
    y=precip_climatology,
    name='Precipitation',
    # Set the y-axis to be on the right side
    yaxis='y2'
))

# Add a line chart for temperature
fig.add_trace(go.Scatter(
    x=temp_climatology.month,
    y=temp_climatology,
    mode='lines',
    name='Temperature',
))

# Set the layout to have two y-axes
fig.update_layout(
    yaxis=dict(title='Temperature'),
    yaxis2=dict(title='Precipitation', overlaying='y', side='right')
)

# Show the figure
fig.show()